<a href="https://colab.research.google.com/github/sadra-barikbin/persian-sentiment-analysis-example/blob/main/sentiment-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install clean-text[gpl]

In [1]:
import numpy as np
import pandas as pd
import torch
import cleantext
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler

# Loading & Preparing Data

In [ ]:
!gdown 'https://drive.google.com/uc?id=1HH8QFDcvkKfnj4dWmFQceb3PpNqDD8HQ&authuser=0&export=download'
!gdown 'https://drive.google.com/uc?id=1uDOO8RP7Lr9qcRJO8z3d10qm_UggJv4I&authuser=0&export=download'

In [2]:
train_df = pd.read_csv('train.csv')
eval_df = pd.read_csv('eval.csv')

In [3]:
train_df.head()

,Unnamed: 0,comment,rate
0,2587,پردازنده های Core i5 و Core i3 نیز ذاتا دو هست...,0.0
1,22591,سلام به دوستای عزیزم \nعزاداری هاتون قبول باشه,1.0
2,141037,کلا پولتون رو دور نریزیزد,-1.0
3,58593,از صمیم قلب امیدوارم دایانا با کارن بمونه و پو...,1.0
4,5712,آنطور که اپل ادعا می کند آیپاد شافل دارای طراح...,1.0


In [4]:
eval_df.head()

,Unnamed: 0,comment,rate
0,61591,کیفیت غذا و زمان رسیدن عالی بود,-1.0
1,50299,در‌ حد ساندویچ یه نفره بود نه دونفره یا بمب. ک...,1.0
2,2777,طعم پیتزای چهار فصل مثل همشه خیلی خوب بود اما ...,-1.0
3,9126,مشخصات سخت افزاری مناسب در کنار سیستم عامل وین...,0.5
4,7544,مرغش سوخاری و خوشمزه بود، بسته بندی عالی، قیمت...,-1.0


In [5]:
train_df = train_df.drop(train_df.columns[0], axis=1)
eval_df = eval_df.drop(eval_df.columns[0], axis=1)

In [6]:
print(f"{len(train_df[train_df.rate == 0])} out of {len(train_df)} train comments have rate zero.")
print(f"{len(eval_df[eval_df.rate == 0])} out of {len(eval_df)} eval comments have rate zero.")

104 out of 800 train comments have rate zero.
30 out of 200 eval comments have rate zero.


In [7]:
train_df = train_df[train_df.rate != 0]
eval_df  = eval_df[eval_df.rate != 0]

In [8]:
train_df['rate'] = train_df.rate.apply(lambda r: 1 if r > 0 else 0)
eval_df['rate'] = eval_df.rate.apply(lambda r: 1 if r > 0 else 0)

## Balancing Dataset
As you can see below, data is imbalanced. We use over-sampling strategy on negative class to mitigate the problem.

In [9]:
pd.concat([train_df.rate.value_counts().rename('train'),
           eval_df.rate.value_counts().rename('eval')], axis=1)

,train,eval
1,502,115
0,194,55


In [10]:
balancer = RandomOverSampler(random_state=41)
train_df, _ = balancer.fit_resample(train_df, train_df.rate)
eval_df, _ = balancer.fit_resample(eval_df, eval_df.rate)

## Normalization

In [11]:
params = {'to_ascii':False, 'no_urls':True,    'no_phone_numbers':True, 'no_line_breaks':True,
          'no_emails':True, 'no_numbers':True, 'no_digits':True,        'no_currency_symbols':True}

train_df['comment'] = train_df.comment.apply(lambda c: cleantext.clean(c,**params))
eval_df['comment'] = eval_df.comment.apply(lambda c: cleantext.clean(c,**params))

## Vocabulary

In [12]:
with open("vocab.txt") as fp:
    words = set([w.strip() for w in fp.readlines()])
len(words)

60271

In [111]:
from nltk.tokenize import word_tokenize

for c in train_df['comment']:
    words.update(word_tokenize(c))

In [13]:
vocab = sorted(words)

In [113]:
with open('vocab.txt', 'w') as fw:
    for w in vocab:
        fw.write(w + '\n')

In [114]:
len(vocab)

60273

# Method 1: Linear Models
We make use of Logistic Regression and SVM as classifiers, and for vectorizing the comments, Tfidf is used.

In [38]:
vectorizer = TfidfVectorizer(max_features=600, ngram_range=(1,3))

In [39]:
pipeline = Pipeline([('embedding', vectorizer),
                     ('classifier', LinearSVC())])
pipeline.fit(train_df.comment, train_df.rate)

Pipeline(steps=[('embedding',
                 TfidfVectorizer(max_features=600, ngram_range=(1, 3))),
                ('classifier', LinearSVC())])

## Hyper-parameter Tuning
We search over different settings and find the best.

In [34]:
param_grid = {'embedding__ngram_range': [(1,2),(1,3),(1,4)],
              'embedding__max_features': range(100, 3000, 100),
              'classifier': [LinearSVC(),LogisticRegression()]}

In [35]:
# The smelling code here is due to Scikit GridSearchCV's specific input for `cv` parameter.
# GridSearchCV and other meta-estimators in Scikit accept whole data (train+eval) in their `fit`
# method. So if you have a dataset separated in train and eval parts beforehand, you should
# concatenate them. Beside that you have to give indices of train and eval parts as the `cv` parameter.

train_eval = pd.concat((train_df, eval_df), ignore_index=True)
train_eval_indices = [(train_df.index, eval_df.index + len(train_df))]
meta_estimator = GridSearchCV(pipeline, param_grid, scoring=['accuracy', 'f1'],
                              cv=train_eval_indices, refit='f1', verbose=1)
_ = meta_estimator.fit(train_eval.comment, train_eval.rate)

Fitting 1 folds for each of 174 candidates, totalling 174 fits


In [36]:
meta_estimator.best_params_

{'classifier': LinearSVC(),
 'embedding__max_features': 600,
 'embedding__ngram_range': (1, 3)}

In [37]:
print(f"Best model F1: {meta_estimator.best_score_}")

Best model F1: 0.7063197026022304


## Determining Marker Features

In [43]:
selector = SelectFromModel(pipeline, threshold=-np.inf, max_features=10, prefit=True,
                           importance_getter='named_steps.classifier.coef_')

pipeline['embedding'].get_feature_names_out()[selector.get_support()]

array(['ارسال', 'اینکه', 'لک', 'معمولی', 'من خیلی', 'میدان', 'نيست',
       'هم که', 'چندان', 'کاملا'], dtype=object)

# Method 2: Neural Networks

In [14]:
import tensorflow as tf

In [15]:
EMBED_DIM = 200
SEQ_LEN = 64
VOCAB_SIZE = len(vocab)
BATCH_SIZE = 20
CLASS_NO = 2

In [16]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df['comment'], train_df['rate'])).shuffle(buffer_size=len(train_df)).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
eval_ds = tf.data.Dataset.from_tensor_slices((eval_df['comment'], eval_df['rate'])).shuffle(buffer_size=len(eval_df)).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [20]:
from tensorflow.keras.layers import TextVectorization, Embedding, Input, Dense, Bidirectional, LSTM
from tensorflow.keras.models import Sequential

def create_lstm_model(lstm_dim=64, dense_dim=16):
    vectorizer = TextVectorization(vocabulary='vocab.txt',
                                   output_mode='int', 
                                   output_sequence_length=SEQ_LEN)
    return Sequential([
        Input(shape=(1,), dtype=tf.string),
        vectorizer,
        Embedding(VOCAB_SIZE + 4, EMBED_DIM, name='embedding'),
        Bidirectional(LSTM(lstm_dim)),
        Dense(dense_dim, activation='relu'),
        Dense(CLASS_NO, 'softmax')
    ])

In [21]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [22]:
model = create_lstm_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', f1_m])

In [25]:
len(model.layers[0].get_vocabulary())

60275

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 64)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 64, 200)           12055000  
                                                                 
 bidirectional (Bidirectiona  (None, 128)              135680    
 l)                                                              
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dense_1 (Dense)             (None, 2)                 34        
                                                                 
Total params: 12,192,778
Trainable params: 12,192,778
No

In [27]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy', mode='max')

In [28]:
model.fit(train_ds, validation_data=eval_ds, epochs=10)

Epoch 1/10
51/51 [==============================] - 14s 207ms/step - loss: 0.6790 - accuracy: 0.5916 - f1_m: 0.6463 - val_loss: 0.6745 - val_accuracy: 0.6000 - val_f1_m: 0.6604
Epoch 2/10
51/51 [==============================] - 10s 195ms/step - loss: 0.3515 - accuracy: 0.8855 - f1_m: 0.6558 - val_loss: 0.9430 - val_accuracy: 0.6130 - val_f1_m: 0.6616
Epoch 3/10
51/51 [==============================] - 10s 191ms/step - loss: 0.0733 - accuracy: 0.9801 - f1_m: 0.6623 - val_loss: 1.2968 - val_accuracy: 0.6130 - val_f1_m: 0.6530
Epoch 4/10
51/51 [==============================] - 10s 189ms/step - loss: 0.0254 - accuracy: 0.9920 - f1_m: 0.6569 - val_loss: 1.3554 - val_accuracy: 0.6261 - val_f1_m: 0.6655
Epoch 5/10
51/51 [==============================] - 10s 193ms/step - loss: 0.0131 - accuracy: 0.9960 - f1_m: 0.6586 - val_loss: 1.8099 - val_accuracy: 0.6304 - val_f1_m: 0.6607
Epoch 6/10
51/51 [==============================] - 10s 193ms/step - loss: 0.0112 - accuracy: 0.9960 - f1_m: 0.6631

# Method 3: Pre-trained Language Models